In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.001

Inductance = 1 #5.
R = 1.

n_func = 2
# solver = "ca"
solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "cvx_dccp"
# solver = "qcqp_sni"
# solver = "bisection_b"

# Ciropt problem

In [4]:
# problem = co.admm_consensus2(mu, L_smooth, R, Inductance)
# problem = co.admm_consensus(n_func, mu, L_smooth, R, Inductance)
problem = co.admm_consensus_pep(mu, L_smooth, R, Inductance)
problem.obj = problem.b + problem.d

# bounds = {  'b': {"ub": 0.1, "lb": 0.1},
#             'h': {"ub": 1, "lb": 0},
#             'alpha': {"ub": 1, "lb": 1},
#             'beta': {"ub": 1, "lb": 1},}

# bounds = {  'b': {"ub": 10, "lb": 0.},
#             'd': {"ub": 0, "lb": 0.},
#             'h': {"ub": 10, "lb": 0},
#             'alpha': {"ub": 1, "lb": -1},
#             'beta': {"ub": 1, "lb": -1},}

res, model = problem.solve(solver=solver, verbose=True)[:2]
ca_vars = problem.vars

# res = params = {  'b': 10,
#             'h': 0.0001,
#             'd': 0.,
#             'alpha': 0.5,
#             'beta': 0.5 }
# b = problem.solve(solver=solver, freq=1, max_iters=30, params=params, debug=True)
# print(f"{b = }")
# res["b"] = b

dim_G=7, dim_F=6

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1655
Number of nonzeros in inequality constraint Jacobian.:      213
Number of nonzeros in Lagrangian Hessian.............:      304

Total number of variables............................:       73
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       61
Total numbe

In [5]:
res

{'b': 1.001956775390173,
 'h': 0.9999044725934395,
 'd': 5.99998145290392,
 'alpha': 0.0,
 'beta': 0.0}

# PEP verification

In [6]:
# res = {'b': 1,
#         'h': 1,
#         'd': 0.,
#         'alpha': 0,
#         'beta': 1}

In [9]:
# problem = co.admm_consensus_pep(n_func, mu, L_smooth, R, Inductance, params=res)
problem = co.admm_consensus_pep(mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 7x7
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (2 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
		 function 2 : Adding 6 scalar constraint(s) ...
		 function 2 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: -2.220136031613751e-09
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -7.38e-09 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be dee